## Select and Train a Model

In the last two lectures, you got the data and explored it, you sampled a training set and a test set, and you wrote transformation pipelines to clean up and prepare your data for Machine Learning algorithms automatically. You are now ready to select and train a ML model.

In [ ]:
## The following code are from last two lectures
## where we split the data into test and training sets,
## and perform data cleaning/transformation.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

ca_housing = "https://raw.githubusercontent.com/csbfx/advpy122-data/master/California_housing.csv"
housing = pd.read_csv(ca_housing)

########################################
#### Prepare test and training sets ####
########################################

# Split the original dataset into test and training set. Set aside the test set.
# Do this before performing any imputation/transformation
housing["income_cat"] = pd.cut(housing["median_income"],
                              bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
                              labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Now we should remove the income_cat attribute so that
# the data is back to its original state
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Let's separate the predictors and the labels since we don't necessarily want to apply
# the same transformations to the predictors and the target values.
# Let's create a copy of the data and not affect strat_train_set:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

########################
####  Data Cleaning ####
########################

# 1. Handle numerical attributes
imputer = SimpleImputer(strategy="median")
# remove the text attribute because median can only be calculated on numerical attributes:
housing_num = housing.drop('ocean_proximity', axis=1)
# fit the `imputer` instance to the training data using the `fit()` method:
imputer.fit(housing_num)
# Transform the training set:
X = imputer.transform(housing_num)

# 2. Handle text/categorical attributes
housing_cat = housing[['ocean_proximity']]
# convert categorical values into one-hot vectors
cat_encoder = OneHotEncoder() # Adding information: handle_unknowns parameter
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

# 3. Create Custom Transformer

# get the right column indices: safer than hard-coding indices 3, 4, 5, 6
# using list comprehension
rooms_idx, bedrooms_idx, population_idx, household_idx = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

# This is our custom transformer class
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room

    def fit(self, X, y=None):
        return self  # nothing else to do

    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_idx] / X[:, household_idx]
        population_per_household = X[:, population_idx] / X[:, household_idx]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_idx] / X[:, rooms_idx]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

###################################
#### Put together the pipeline ####
###################################

# Pipeline for numerical attributes
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")), # fill in missing values with median values
        ('attribs_adder', CombinedAttributesAdder()),  # add combined attributes to the data
        ('std_scaler', StandardScaler()),              # feature scaling
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)


# Complete the pipeline with combine categorical attributes
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)
cols=list(housing.columns) + ["rooms_per_household", "population_per_household"] + \
    cat_encoder.categories_[0].tolist()

## Convert the 2D array into Pandas DataFrame
housing_prep_df = pd.DataFrame(
    housing_prepared,
    columns=cols,
    index=housing.index)
housing_prep_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household,<1H OCEAN,INLAND,ISLAND,NEAR BAY,NEAR OCEAN
12655,-0.941350,1.347438,0.027564,0.584777,0.640371,0.732602,0.556286,-0.893647,0.017395,0.006223,-0.121122,0.0,1.0,0.0,0.0,0.0
15502,1.171782,-1.192440,-1.722018,1.261467,0.781561,0.533612,0.721318,1.292168,0.569256,-0.040811,-0.810867,0.0,0.0,0.0,0.0,1.0
2908,0.267581,-0.125972,1.220460,-0.469773,-0.545138,-0.674675,-0.524407,-0.525434,-0.018024,-0.075371,-0.338273,0.0,1.0,0.0,0.0,0.0
14053,1.221738,-1.351474,-0.370069,-0.348652,-0.036367,-0.467617,-0.037297,-0.865929,-0.595140,-0.106803,0.961205,0.0,0.0,0.0,0.0,1.0
20496,0.437431,-0.635818,-0.131489,0.427179,0.272790,0.374060,0.220898,0.325752,0.251241,0.006109,-0.474513,1.0,0.0,0.0,0.0,0.0


This is where we last left off. Let's train and evaluate using the training set.

### Training and Evaluating on the Training Set
We have done all the heavy lifting in the previous steps, things are now going to be much simpler.
- Let's first train a Linear Regression model.

In [ ]:
from sklearn.linear_model import LinearRegression

# train a linear regression model with our housing_prepared training data
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

- You now have a working Linear Regression model. Let's try it out on a few instances from the training set:

In [ ]:
# let's try the full preprocessing pipeline on a few training instances
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))

Predictions: [ 85657.90192014 305492.60737488 152056.46122456 186095.70946094
 244550.67966089]


In [ ]:
some_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
12655,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND
15502,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN
2908,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.8750,INLAND
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN
20496,-118.70,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN


- Compare against the actual values:

In [ ]:
print("Labels:", list(some_labels))

Labels: [72100.0, 279600.0, 82700.0, 112500.0, 238300.0]


- Let's select a performance measure to evaluate the performance of our ML model
- A typical performance measure for regression programs is **Root Mean Square Error (RMSE)**. It gives an idea of how much error the system typically makes in its prediction, with a higher weight for large errors.
- We measure this regression model's RMSE on the whole training set using Scikit-Learn's [`mean_squared_error`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) function.

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)

# sklearn.metrics.mean_squared_error(y_true, y_pred)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
# root mean squared error
lin_rmse = np.sqrt(lin_mse)
print("lin_rmse:", lin_rmse)

# We can also employ another performance measure called
# Mean Asolute Error
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_labels, housing_predictions)
print("lin_mae:", lin_mae)

lin_rmse: 68627.87390018745
lin_mae: 49438.66860915803


This shows that our model typical prediction error is \\$68,628 according to the RMSE. This is an example of model underfitting the training data.

**Reasons for underfitting**:
1. The features do not provide enough information to make good predictions, or
2. The model is not powerful enough

**The main ways to fix underfitting are to:**
1. select a more powerful model, or
2. to feed the training algorithm with better feature, or
3. to reduce the constrains on the model.

Our model is not regularized, so this rules out the last option. We could try add more features (e.g. the log of the population), but first let's try a more complex model to see how it does.

- Let's train a `DecisionTreeRegressor`. This is a powerful model, capable of finding complex non-linear relationship in the data.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(random_state=42)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

No error at all?!?! Could this model really be absolutely perfect? Of course it is much more likely that this model has badly overfit the data. But how can you be sure? As we saw earlier, we don't want to touch the test set until we are ready to launch a model that we are confident about, so we need to use part of the training set for training, and part for model validation.

#### Better Evaluation Using Cross_Validation ####

We can use Scikit-Learn's `cross-validation` feature to perform *K-fold cross validation*: it randomly splits the training set into 10 distinct subsets called *folds*, then it trains and evaluates the Decision Tree model 10 times, picking a different fold for evaluation every time and train on the other 9 folds. The result is an array containing the 10 evaluation scores:

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                        scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

Scores: [72831.45749112 69973.18438322 69528.56551415 72517.78229792
 69145.50006909 79094.74123727 68960.045444   73344.50225684
 69826.02473916 71077.09753998]
Mean: 71629.89009727491
Standard deviation: 2914.035468468928


Now the Decision Tree doesn't look as good as it did earlier. In fact, it seems to perform worse than the Linear Regression model. Notice that the cross-validation allows you to get not only an estimate of the performance of your model, but also a measure of how precise this estimate is (i.e., its standard deviation). The  Decision Tree has a score of approximately 71,407, generally ± 2439.

Let's compute the same scores for Linear Regression model just to be sure:

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                        scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

display_scores(lin_rmse_scores)

Scores: [71762.76364394 64114.99166359 67771.17124356 68635.19072082
 66846.14089488 72528.03725385 73997.08050233 68802.33629334
 66443.28836884 70139.79923956]
Mean: 69104.07998247063
Standard deviation: 2880.3282098180634


That's right: The Decision Tree model is overfitting so badly that it performs worse than Linear Regression model.

Let's try one last model: the `RandomForestRegression`. Random Forests work by training many Decision Trees on random subsets of the features, then average out their predictions. Building a model on top of many other models is called *Ensemble Learning*, and it is often a great way to push ML algorithms even further.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

22413.454658589766

In [ ]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [53519.05518628 50467.33817051 48924.16513902 53771.72056856
 50810.90996358 54876.09682033 56012.79985518 52256.88927227
 51527.73185039 55762.56008531]
Mean: 52792.92669114079
Standard deviation: 2262.8151900582


Random Forests look more promising. However, not that the score on the training set is still much lower than on the validation sets, meaning that the model is still overfitting the training set. Possible solution for overfitting are to simplify the model, contrain it, or get a lot more training data.

However, before we dive much deeper in Random Forests, we should try out many other models from various categories of Machine Learning algorithms (several Support Vector Machines with different kernels, possibly a neural network, etc.), without spending too much time tweaking the hyperparameters.

You should save every model you experiement with, so you can come back easily to any model you want. Make sure you save both the hyperparameters and the trained parameters, as well as the cross-validation scores and perhaps the actual predictions as well. We can easily [save Scikit-Learn models](https://scikit-learn.org/stable/modules/model_persistence.html) by using Python's [`pickle`](https://docs.python.org/3/library/pickle.html) module or [`joblib`](https://pypi.org/project/joblib/).

Read more about [Model Evaluation](https://scikit-learn.org/0.15/modules/model_evaluation.html).

## Fine-Tune Your Model
Let's assume we have a shortlist of promising models, and we want to fine-tune them. We can do it in the following ways:

#### 1.  Grid Search
One way to do that would be fiddle with the hyperparameters manually, until you find a great combination of hyperparameter values. This would be **very tedious** work, and you may not have time to explore many combinations.

Instead, you should use Scikit-Learn's `GridSearchCV` to search for you. All you need to do is tell it which hyperparameters you want it to experiment with, and what values to try out, and it will evaluate all the possible combinations of hyperparameter values, using cross-validation. For example, the following code searches for the best combination of hyperparameter values for the `RandomForestRegressor`:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

The `param_grid` tells Scikit-Learn to first evaluate all 3 x 4 = 12 combinations of `n_estimator` and `max_features` hyperparameter values specificed in the first dictionary, then try all 2 x 3 = 6 combinations of hyperparameter values in the second dictionary, but this time with the `bootstrap` hyperparameter set to False (which is True by default).

All in all, the grid search will explore 12 + 6 = 18 combinations of `RandomForestRegressor` hyperparameter values, and it will train each model five times (since we set `cv=5` in `GridSearchCV`). In other words, all in all, there will be 18 x 5 = 90 rounds of training! It may take quite a long time, but when it is done, you can get the best combination of parameter like this:

In [ ]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

Since 8 and 30 are the maximum values that were evaluated, you should probably try searching again with higher values, since the score may continue to improve.

We can also get the best estimator directly:

In [ ]:
print(grid_search.best_estimator_)

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)


The evaluation scores are also available. Let's look at the score of each hyperparameter combination tested during the grid search:

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

63895.161577951665 {'max_features': 2, 'n_estimators': 3}
54916.32386349543 {'max_features': 2, 'n_estimators': 10}
52885.86715332332 {'max_features': 2, 'n_estimators': 30}
60075.3680329983 {'max_features': 4, 'n_estimators': 3}
52495.01284985185 {'max_features': 4, 'n_estimators': 10}
50187.24324926565 {'max_features': 4, 'n_estimators': 30}
58064.73529982314 {'max_features': 6, 'n_estimators': 3}
51519.32062366315 {'max_features': 6, 'n_estimators': 10}
49969.80441627874 {'max_features': 6, 'n_estimators': 30}
58895.824998155826 {'max_features': 8, 'n_estimators': 3}
52459.79624724529 {'max_features': 8, 'n_estimators': 10}
49898.98913455217 {'max_features': 8, 'n_estimators': 30}
62381.765106921855 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
54476.57050944266 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59974.60028085155 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52754.5632813202 {'bootstrap': False, 'max_features': 3, 'n_estimators': 1

In this example, we obtain the best solution by setting the `max_features` hyperparameter to 8, and the `n_estimators` hyperparameter to 30. The RMSE score for this combination is 49,682, which is slightly better than the score we got earlier using default hyperparameter values (which was 50,182). Yay! We have successfully fine-tuned our best model.

#### 2. Randomized Search
The grid search approach is fine when you are exploring relatively few combinations, like in the previous example, but when the hyperparameter *search space* is large, it is often preferable to use `RandomizedSearchCV` instead. This class can be used in much the same way as the `GridSearchCV` class, but instead of trying out all possible combinations, it evaluates a given number of random combinations by selecting a random value for each hyperparameter at every iteration. This approach has two main benefits:
1. If you let randomized search run for, say, 1,000 iterations, this approach will explore 1,000 different values for each hyperparameter (instead of just a few values per hyperparameter with the grid search approach).
2. You have more control over the computing budget you want to allocate to hyperparameter search, simply by setting the number of iterations.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error',
                                random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79c4148d6770>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79c4148d7730>},
                   random_state=42, scoring='neg_mean_squared_error')

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

49117.55344336652 {'max_features': 7, 'n_estimators': 180}
51450.63202856348 {'max_features': 5, 'n_estimators': 15}
50692.53588182537 {'max_features': 3, 'n_estimators': 72}
50783.614493515 {'max_features': 5, 'n_estimators': 21}
49162.89877456354 {'max_features': 7, 'n_estimators': 122}
50655.798471042704 {'max_features': 3, 'n_estimators': 75}
50513.856319990606 {'max_features': 3, 'n_estimators': 88}
49521.17201976928 {'max_features': 5, 'n_estimators': 100}
50302.90440763418 {'max_features': 3, 'n_estimators': 150}
65167.02018649492 {'max_features': 5, 'n_estimators': 2}


#### 3. Ensemble Methods
Another way to fine-tune your system is to try to combine the models that perform best. The group (or "ensemble") will often perform better than the best individual model (just like Random Forests perform better than the individual Decision Trees they rely on), especially if the individual models make very different types of errors.

## Analyze the Best Models and Their Errors
We will often gain good insights on the problem by inspecting the best models. For example, the `RandomForestRegressor` can indicate the relative importance of each attribute for making accurate predictions:

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([6.96542523e-02, 6.04213840e-02, 4.21882202e-02, 1.52450557e-02,
       1.55545295e-02, 1.58491147e-02, 1.49346552e-02, 3.79009225e-01,
       5.47789150e-02, 1.07031322e-01, 4.82031213e-02, 6.79266007e-03,
       1.65706303e-01, 7.83480660e-05, 1.52473276e-03, 3.02816106e-03])

Let's display these important scores next to their corresponding attribute names:

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.3790092248170967, 'median_income'),
 (0.16570630316895876, 'INLAND'),
 (0.10703132208204354, 'pop_per_hhold'),
 (0.06965425227942929, 'longitude'),
 (0.0604213840080722, 'latitude'),
 (0.054778915018283726, 'rooms_per_hhold'),
 (0.048203121338269206, 'bedrooms_per_room'),
 (0.04218822024391753, 'housing_median_age'),
 (0.015849114744428634, 'population'),
 (0.015554529490469328, 'total_bedrooms'),
 (0.01524505568840977, 'total_rooms'),
 (0.014934655161887776, 'households'),
 (0.006792660074259966, '<1H OCEAN'),
 (0.0030281610628962747, 'NEAR OCEAN'),
 (0.0015247327555504937, 'NEAR BAY'),
 (7.834806602687504e-05, 'ISLAND')]

With this information, we may want to try dropping some of the less useful features (e.g. apparently only one `ocean_proximity` category is really useful - 'INLAND', so we could try dropping the others).

We should also look at the specific errors that our sytem makes, then try to understand why it makes them, and what could fix the problem (adding extra features, or getting rid of uninformative ones, cleaning up outliers, etc.).

## Evaluate Your System on Test Set

After tweaking our models for a while, we finally have a system that perform sufficiently well. Now is the time to evaluate the final model on the test set. There is nothing sepcial about this process; just get the predictors and the labels from our test set, run our full_pipeline to transform the data (call `tranform()`, **not `fit_transform()`**, we don't want to fit the test set!), and evaluate the final model on the test set:

In [ ]:
final_model = grid_search.best_estimator_
# final_model
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

47873.26095812988

The performance will usually be slightly worse than what you measured using cross-validation if you did a lot of hyperparameter tuning (because your system ends up fine-tuned to perform well on the validation dta, and will likely not perform as well on unknown datasets). It is not the case in this example. However, when this happens, **you must resist the tempatation to tweak the hyperparameters to make the numbers look good on the test set**; the improvements would be unlikely to generalize to new data.

## Present your results
Before launching the system, you need to present your solution (highlight what you have learned, what worked, and what did not, what assumptions were made, and what are the limitations of your system. Document everything, and create nice presentations with clear visualizations and easy-to-remember statements (e.g., **"the median income is the number one predictor of housing prices"**).